In [1]:
from nltk.corpus import gutenberg

In [7]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

train_data = [({"a": 4, "b": 1, "c": 0}, "ham"),
                  ({"a": 5, "b": 2, "c": 1}, "ham"),
                  ({"a": 0, "b": 3, "c": 4}, "spam"),
                  ({"a": 5, "b": 1, "c": 1}, "ham"),
                  ({"a": 1, "b": 4, "c": 3}, "spam")]

classif = SklearnClassifier(BernoulliNB()).train(train_data)
test_data = [{"a": 3, "b": 2, "c": 1},
             {"a": 0, "b": 3, "c": 7}]

classif.classify_many(test_data)
#print(classif.classify(gender_features('Frank')))
#classif.show_most_informative_features(5)
print(classif.labels())

classif = SklearnClassifier(SVC(), sparse=False).train(train_data)
classif.classify_many(test_data)

print(classif.labels())

['ham', 'spam']
['ham', 'spam']


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [3]:
import nltk;
nltk.corpus.gutenberg.fileids()


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [7]:
emma = nltk.Text(nltk.corpus.gutenberg.words('austen-emma.txt'))
emma.concordance("pleasure")
len(emma)

Displaying 25 of 115 matches:
y interested in herself , in every pleasure , every scheme of hers -- one to w
ow herself to feel so much pain as pleasure . Every friend of Miss Taylor must
 ." " And have you never known the pleasure and triumph of a lucky guess ?-- I
meet him ." " With a great deal of pleasure , sir , at any time ," said Mr . K
pain , of Emma ' s losing a single pleasure , or suffering an hour ' s ennui ,
the present evening had particular pleasure in sending them away happy . The h
duction had given as much panic as pleasure ; but the humble , grateful little
be plenty of people who would take pleasure in degrading you ." " Yes , to be 
g their intimacy with the greatest pleasure . How very differently we feel !--
 it , so long as it is a source of pleasure to herself . It has been so many y
n the hand !" " Great has been the pleasure , I am sure . I never met with a d
ith the commission , what infinite pleasure should he have in executing it ! h
oodhouse , I would not

192427

In [8]:
text1

NameError: name 'text1' is not defined

In [9]:
nltk.corpus.gutenberg.fileids([1])

TypeError: fileids() takes 1 positional argument but 2 were given

In [11]:
a= nltk.corpus.gutenberg.fileids()

In [12]:
a

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [13]:
a[1]

'austen-persuasion.txt'

In [14]:
a[1:3]

['austen-persuasion.txt', 'austen-sense.txt']

In [15]:
a[0:3]

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt']

In [16]:
b= a[0:3]

In [17]:
b

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt']

In [18]:
b=a[11:14]

In [19]:
b

['edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt']

In [20]:
len(b)

3

In [4]:
 m = re.search('(?<=-)\w+', 'spam-egg')
m.group(0)

'egg'

In [29]:
print(gutenberg.raw('austen-emma.txt')[0:1000])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died too long ago for her to have more than an indistinct
remembrance of her caresses; and her place had been supplied
by an excellent woman as governess, who had fallen little short
of a mother in affection.

Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
less as a governess than a friend, very fond of both daughters,
but particularly of Emma.  Between _them_ it was more the intimacy
of sisters.  Even before Miss Taylor had ceased to hold the nominal
office of governess, the mildness o

In [31]:
len(b[1])

22

In [43]:
c = nltk.Text(nltk.corpus.gutenberg.words('austen-emma.txt'));
len(c)

192427

In [48]:
a= nltk.Text(nltk.corpus.gutenberg.words('edgeworth-parents.txt'));
b= nltk.Text(nltk.corpus.gutenberg.words('melville-moby_dick.txt'));
c= nltk.Text(nltk.corpus.gutenberg.words('milton-paradise.txt'));
len(a)


210663

In [51]:
len(c)

96825

In [53]:
print(gutenberg.raw('edgeworth-parents.txt')[0:145200])

[The Parent's Assistant, by Maria Edgeworth]


THE ORPHANS.

Near the ruins of the castle of Rossmore, in Ireland, is a small cabin,
in which there once lived a widow and her four children.  As long as she
was able to work, she was very industrious, and was accounted the best
spinner in the parish; but she overworked herself at last, and fell ill,
so that she could not sit to her wheel as she used to do, and was obliged
to give it up to her eldest daughter, Mary.

Mary was at this time about twelve years old.  One evening she was
sitting at the foot of her mother's bed spinning, and her little brothers
and sisters were gathered round the fire eating their potatoes and milk
for supper.  "Bless them, the poor young creatures!" said the widow, who,
as she lay on her bed, which she knew must be her deathbed, was thinking
of what would become of her children after she was gone.  Mary stopped
her wheel, for she was afraid that the noise of it had wakened her
mother, and would hinder her from

In [55]:
import random
random.sample(gutenberg.raw('edgeworth-parents.txt'),145200)

['h',
 'p',
 ' ',
 'r',
 ' ',
 'k',
 'i',
 'l',
 'h',
 ' ',
 't',
 ' ',
 'h',
 'b',
 'h',
 'b',
 'u',
 'e',
 'a',
 'i',
 'n',
 'f',
 'i',
 'a',
 'd',
 'p',
 'r',
 'o',
 'm',
 ' ',
 'o',
 ' ',
 'f',
 'n',
 'r',
 'h',
 'h',
 'o',
 's',
 't',
 'a',
 ' ',
 'e',
 ' ',
 '!',
 ' ',
 't',
 ' ',
 'w',
 'i',
 'k',
 'a',
 'e',
 'd',
 'c',
 'g',
 ' ',
 '\r',
 's',
 'h',
 's',
 'a',
 's',
 'e',
 'c',
 't',
 'a',
 't',
 't',
 ' ',
 'i',
 ' ',
 'h',
 'e',
 'y',
 'e',
 'l',
 'c',
 'o',
 'i',
 'o',
 'I',
 'b',
 ' ',
 'e',
 't',
 'o',
 '\n',
 's',
 't',
 ' ',
 'd',
 'u',
 'd',
 'H',
 'i',
 'e',
 'l',
 'u',
 'r',
 'h',
 'f',
 'e',
 'a',
 ' ',
 't',
 'e',
 'i',
 'i',
 '\r',
 'a',
 '\r',
 'o',
 't',
 'e',
 'a',
 'i',
 '.',
 'd',
 's',
 '\n',
 ' ',
 't',
 '\n',
 's',
 'h',
 ' ',
 'e',
 'n',
 'w',
 'e',
 'o',
 '"',
 ' ',
 'f',
 'n',
 't',
 'c',
 ' ',
 'n',
 '\n',
 'n',
 's',
 'n',
 'o',
 's',
 ' ',
 'a',
 's',
 'h',
 'a',
 'V',
 '?',
 ',',
 ' ',
 'o',
 'n',
 'i',
 's',
 ' ',
 ' ',
 ' ',
 'h',
 'c',
 ' ',
 'o